# Data format overview
## In this tutorial, we will overview evaluate the following data formats
1. avro (structured)
2. csv (semi-structured)
3. json (semi-structured)
4. orc (structured)
5. parquet (structured) 

## We evaluate the data formats via:
1. Disk usage
2. Read/Write latency
3. Random data lookup
4. Filtering/GroupBy(column-wise)
5. Distinct(row-wise)

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
       .builder.master("local[2]") \
       .appName("Python Spark SQL basic example") \
       .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.0.1") \
       .getOrCreate()

In [3]:
json_data_path="/home/pliu/git/format-performence-test/netflix.json"
parquet_data_path="/home/pliu/git/format-performence-test/netflix.parquet"
avro_data_path="/home/pliu/git/format-performence-test/netflix.avro"
orc_data_path="/home/pliu/git/format-performence-test/netflix.orc"
csv_data_path="/home/pliu/git/format-performence-test/netflix.csv"

# Some useful functions for evaluating data format

### The read function read the source data file and convert it to a spark data frame

In [4]:
data_format_stats_path="../tmp/mystats.csv"
def write_stats(line):
    file1 = open(data_format_stats_path,"a")
    file1.write(line+"\n")

In [5]:
import time
def read(fmt):
    start = time.time()
    if fmt == "json":
        sdf = spark.read.option("header", "true").json(json_data_path)
    elif fmt == "csv":
        sdf = spark.read.option("header", "true").csv(csv_data_path)
    elif fmt == "avro":
        sdf = spark.read.format("avro").load(avro_data_path)
    elif fmt == "parquet":
        sdf = spark.read.parquet(parquet_data_path)
    elif fmt == "orc":
        sdf = spark.read.orc(orc_data_path)
    sdf.show(5,False)
    stats="{}, {}, {}".format(fmt, "read", time.time() - start)
    write_stats(stats)
    print(stats)
    return sdf

### The get_shape function prints the shape(e.g. row number and column number) of the data frame

In [6]:
def get_shape(df,fmt):
    start = time.time()
    row_num=df.count()
    col_num=len(df.columns)
    stats="{}, {}, {}".format(fmt, "get_shape", time.time() - start)
    write_stats(stats)
    print("The data frame has {} rows and {} columns".format(row_num,col_num))
    print(stats)

### The stats function prints the min, max and numbers of a column of the data frame

In [7]:
def stats(df,fmt, field="rating"):
    start = time.time()
    max=df.agg({field: "max"})
    min=df.agg({field: "min"})
    count=df.agg({field: "count"})
    min.show(5,False)
    max.show(5,False)
    count.show(5,False)
    stats="{}, {}, {}".format(fmt, "stats", time.time() - start)
    write_stats(stats)
    print(stats)

### The random_batch function randomly select rows from the data frame. It can evaluate the ability of random data lookup

In [8]:
def random_batch(df,fmt):
    start = time.time()
    result=df.sample(False, 0.05).collect()
    stats="{}, {}, {}".format(fmt, "random_batch", time.time() - start)
    write_stats(stats)
    print(stats)
   # return result

### The distinct function count distinct rows of the data frame

In [9]:
def distinct(df,fmt):
    start = time.time()
    result = df.distinct().count()
    stats="{}, {}, {}".format(fmt, "distinct", time.time() - start)
    write_stats(stats)
    print(stats)
    return result

### The group_by function group and count the data frame by a specific column

In [10]:
def group_by(df,fmt):
    start = time.time()
    result=df.groupBy("rating").count()
    result.show(5,False)
    stats="{}, {}, {}".format(fmt, "group_by", time.time() - start)
    write_stats(stats)
    print(stats)
    #return result

### The filtering function filter data by using a specific boolean condition

In [11]:
def filtering(df, fmt, date="2005-11-15"):
    start = time.time()
    result = df.filter(df.date > date).count()
    stats="{}, {}, {}".format(fmt, "filtering", time.time() - start)
    write_stats(stats)
    print(stats)
    return result

In [12]:
def saveCSV(df,outputPath,fileName): Unit ={
    df.coalesce(1).write.mode("overwrite")
      .option("header","true")
      .option("mapreduce.fileoutputcommitter.marksuccessfuljobs","false")
      .option("encoding", "UTF-8")
      .option("delimiter", ",") 
      .csv(outputPath+"/"+fileName)
  }   

# 1. Gathering stats of each data format

## 1.1 Get CSV format evaluation stats

In [13]:
csv_df=read("csv")

+-------+------+----------+
|user_id|rating|date      |
+-------+------+----------+
|1488844|3     |2005-09-06|
|822109 |5     |2005-05-13|
|885013 |4     |2005-10-19|
|30878  |4     |2005-12-26|
|823519 |3     |2004-05-03|
+-------+------+----------+
only showing top 5 rows

csv, read, 4.87298059463501


In [14]:
csv_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- date: string (nullable = true)



In [15]:
get_shape(csv_df,"csv")

The data frame has 24058262 rows and 3 columns
csv, get_shape, 8.3622145652771


In [16]:
# get min, max and row number of column rating
stats(csv_df,"csv",field="rating")

+-----------+
|min(rating)|
+-----------+
|1          |
+-----------+

+-----------+
|max(rating)|
+-----------+
|5          |
+-----------+

+-------------+
|count(rating)|
+-------------+
|24053764     |
+-------------+

csv, stats, 26.145728588104248


In [17]:
random_batch(csv_df,"csv")

csv, random_batch, 15.357799291610718


In [18]:
distinct(csv_df,"csv")

csv, distinct, 19.20894479751587


12168704

In [19]:
group_by(csv_df,"csv")

+------+-------+
|rating|count  |
+------+-------+
|3     |6904181|
|null  |4498   |
|5     |5506583|
|1     |1118186|
|4     |8085741|
+------+-------+
only showing top 5 rows

csv, group_by, 8.280380249023438


In [20]:
filtering(csv_df,"csv")

In [28]:
name="netflix"
csv_df.write.mode("overwrite").option("header", "true").csv("{}.csv".format(name))

## 1.2 Get Json format evaluation stats

In [98]:
json_df=read("json")

+----------+------+-------+
|date      |rating|user_id|
+----------+------+-------+
|2005-09-06|3     |1488844|
|2005-05-13|5     |822109 |
|2005-10-19|4     |885013 |
|2005-12-26|4     |30878  |
|2004-05-03|3     |823519 |
+----------+------+-------+
only showing top 5 rows

json, read, 9.966949701309204


In [100]:
get_shape(json_df,"json")

In [111]:
# get min, max and row number of column rating
stats(json_df,"json",field="rating")

+-----------+
|min(rating)|
+-----------+
|1          |
+-----------+

+-----------+
|max(rating)|
+-----------+
|5          |
+-----------+

+-------------+
|count(rating)|
+-------------+
|24053764     |
+-------------+

json, stats, 35.458019971847534


In [101]:
random_batch(json_df,"json")

json, random_batch, 17.13827896118164


In [102]:
distinct(json_df,"json")

json, distinct, 21.003305673599243


12168704

In [115]:
group_by(json_df,"json")

+------+-------+
|rating|count  |
+------+-------+
|3     |6904181|
|null  |4498   |
|5     |5506583|
|1     |1118186|
|4     |8085741|
+------+-------+
only showing top 5 rows

json, group_by, 12.402800798416138


DataFrame[rating: string, count: bigint]

In [103]:
filtering(json_df,"json")

json, filtering, 11.020655632019043


850269

## 1.3 Get Avro format evaluation stats

In [17]:
avro_df=read("avro")

+-------+------+----------+
|user_id|rating|date      |
+-------+------+----------+
|2584438|4     |2005-03-01|
|702567 |4     |2005-03-05|
|1253149|3     |2005-03-06|
|321108 |4     |2005-03-08|
|254824 |5     |2005-03-11|
+-------+------+----------+
only showing top 5 rows

avro, read, 0.18955612182617188


In [18]:
get_shape(avro_df,"avro")

The data frame has 24058262 rows and 3 columns
json, get_shape, 4.859561204910278


In [19]:
stats(avro_df,"avro",field="rating")

+-----------+
|min(rating)|
+-----------+
|1          |
+-----------+

+-----------+
|max(rating)|
+-----------+
|5          |
+-----------+

+-------------+
|count(rating)|
+-------------+
|24053764     |
+-------------+

avro, stats, 13.65978717803955


In [20]:
random_batch(avro_df,"avro")

avro, random_batch, 9.225749015808105


In [21]:
distinct(avro_df,"avro")

avro, distinct, 29.572277069091797


12168704

In [22]:
group_by(avro_df,"avro")

+------+-------+
|rating|count  |
+------+-------+
|3     |6904181|
|null  |4498   |
|5     |5506583|
|1     |1118186|
|4     |8085741|
+------+-------+
only showing top 5 rows

avro, group_by, 5.662991046905518


In [23]:
filtering(avro_df,"avro")

avro, filtering, 4.33909010887146


850269

## 1.4 Get Parquet format evaluation stats

In [24]:
parquet_df=read("parquet")

+-------+------+----------+
|user_id|rating|date      |
+-------+------+----------+
|2584438|4     |2005-03-01|
|702567 |4     |2005-03-05|
|1253149|3     |2005-03-06|
|321108 |4     |2005-03-08|
|254824 |5     |2005-03-11|
+-------+------+----------+
only showing top 5 rows

parquet, read, 0.936453104019165


In [25]:
get_shape(parquet_df,"parquet")

The data frame has 24058262 rows and 3 columns
parquet, get_shape, 0.1500110626220703


In [106]:
random_batch(parquet_df,"parquet")

parquet, random_batch, 6.159161567687988


In [26]:
stats(parquet_df,"parquet",field="rating")

+-----------+
|min(rating)|
+-----------+
|1          |
+-----------+

+-----------+
|max(rating)|
+-----------+
|5          |
+-----------+

+-------------+
|count(rating)|
+-------------+
|24053764     |
+-------------+

parquet, stats, 3.7586891651153564


In [27]:
distinct(parquet_df,"parquet")

parquet, distinct, 23.636741876602173


12168704

In [28]:
group_by(parquet_df,"parquet")

+------+-------+
|rating|count  |
+------+-------+
|3     |6904181|
|null  |4498   |
|5     |5506583|
|1     |1118186|
|4     |8085741|
+------+-------+
only showing top 5 rows

parquet, group_by, 1.2147870063781738


In [29]:
filtering(parquet_df,"parquet")

parquet, filtering, 0.8171226978302002


850269

# 1.5 Get ORC format evaluation stats

In [130]:
orc_df=read("orc")

+-------+------+----------+
|user_id|rating|date      |
+-------+------+----------+
|1488844|3     |2005-09-06|
|822109 |5     |2005-05-13|
|885013 |4     |2005-10-19|
|30878  |4     |2005-12-26|
|823519 |3     |2004-05-03|
+-------+------+----------+
only showing top 5 rows

orc, read, 2.3085367679595947


In [132]:
get_shape(orc_df,"orc")

The data frame has 24058262 rows and 3 columns
orc, get_shape, 1.4532253742218018


In [133]:
random_batch(orc_df,"orc")

orc, random_batch, 6.4762678146362305


In [134]:
stats(orc_df,"orc",field="rating")

+-----------+
|min(rating)|
+-----------+
|1          |
+-----------+

+-----------+
|max(rating)|
+-----------+
|5          |
+-----------+

+-------------+
|count(rating)|
+-------------+
|24053764     |
+-------------+

orc, stats, 4.612210035324097


In [138]:
distinct(orc_df,"orc")

orc, distinct, 185.58755350112915


12168704

In [135]:
group_by(orc_df,"orc")

+------+-------+
|rating|count  |
+------+-------+
|3     |6904181|
|null  |4498   |
|5     |5506583|
|1     |1118186|
|4     |8085741|
+------+-------+
only showing top 5 rows

orc, group_by, 1.680478811264038


In [136]:
filtering(orc_df,"orc")

orc, filtering, 1.186652421951294


850269

# 2. Visualize the stats of different format 

## 2.1 Visualize the read latency for each format

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
# define a schema
from pyspark.sql.types import StructType, StructField, IntegerType,StringType

schema = StructType([
    StructField("format", StringType(), True),
    StructField("command", StringType(), True),
    StructField("Latency(second)", IntegerType(), True)])


# read stats file
rawDf=spark.read.option("header", "false").csv(data_format_stats_path,schema=schema)
rawDf.show(5)
rawDf.count()

+------+----------+---------------+
|format|   command|Latency(second)|
+------+----------+---------------+
|   csv|      read|           null|
|   csv| get_shape|           null|
|   csv|     stats|           null|
|   csv|     stats|           null|
|   csv|      read|           null|
+------+----------+---------------+
only showing top 5 rows



28

In [36]:
statsDf=rawDf.dropDuplicates(["format","command"])
statsDf.count()

20

# Stop the spark cluster

In [29]:
# stop sparksession
spark.sparkContext.stop()

### Check if the spark cluster is well closed. You should not see any python-spark pods

In [34]:
! kubectl get pods

NAME                                                     READY   STATUS      RESTARTS   AGE
deleting-pods-with-completed-status-1615824000-2nwxn     0/1     Completed   0          58m
jupyter-1615800186-567f67779b-cbmgx                      1/1     Running     0          7h35m
python-spark-sql-basic-example-6e3a6b7836d2d444-exec-1   1/1     Running     0          21s
python-spark-sql-basic-example-6e3a6b7836d2d444-exec-2   1/1     Running     0          21s
python-spark-sql-basic-example-6e3a6b7836d2d444-exec-3   1/1     Running     0          20s
python-spark-sql-basic-example-6e3a6b7836d2d444-exec-4   1/1     Running     0          20s
python-spark-sql-basic-example-6e3a6b7836d2d444-exec-5   1/1     Running     0          20s
ubuntu-1612965548-79c9567b44-nhs9p                       1/1     Running     1          33d
